In [1]:
import pandas as pd
import numpy as np
import requests
import json
import collections
from bs4 import BeautifulSoup
import multiprocessing

In [2]:
BB = pd.read_csv("data/Hot Stuff.csv")
songs = BB["SongID"]; performer = BB["Performer"]
BB_grouped = BB.groupby(["Performer", "Song"]).count()

In [3]:
art_song_lyrics = json.loads(open("genesis_data.json").read())
missing_artists_song = json.loads(open("Genesis_missing.json").read())

In [4]:
song_performer_pair = list(BB_grouped.index)

In [5]:
client_id = "HYxx-SDO43-ap9RUVgD9nNRuz-kmhQw9W-TeCtA4NdmWbLYQUvkCmmcOlNzsdzRE"
client_secred = "2xRkeAd2SDvMLXbkScUzLCZvOPxYXcsdQeiDqhrax-kzxpFkO3fKI4HEKeD5aDlXnYnja7pwYQ9BkMoDvl6ipw"
token = "dxvLAG0Jhx6B01DK9pLzusCIFlboFV996sOvwEAPQbON-MUPz2vLgu9_z7u-hODo"
headers = {'Authorization': "Bearer " + token}
base_url = "http://api.genius.com"
search_url = base_url + "/search"

In [5]:
def get_lyrics(i):
    artist_name, song_title = song_performer_pair[i]
    params = {'q': song_title.lower() + " " + artist_name.lower()}
#     params = {'q': song_title.lower()}
    response = requests.get(search_url, params=params, headers=headers).json()
    song_info = False
    for hit in response["response"]["hits"]:
        if len(set(hit["result"]["primary_artist"]["name"].lower().split(" ")).intersection(set(artist_name.lower().split(" ")))) != 0:
            song_info = hit
            break
    if song_info:
        song_api_path = song_info["result"]["api_path"]
        song_url = base_url + song_api_path
        response = requests.get(song_url, headers=headers).json()
        path = response["response"]["song"]["path"]
        page_url = "http://genius.com" + path
        page = requests.get(page_url)
        html = BeautifulSoup(page.text, "html.parser")
        [h.extract() for h in html('script')]
        lyrics = html.find("div", class_="lyrics").get_text()
        return [artist_name, song_title, [i for i in lyrics.split("\n") if i != ""]]
    else:
        return [False, artist_name, song_title]

In [105]:
art_song_lyrics = collections.defaultdict(dict)
missing_artists_song = collections.defaultdict(set)
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*8) as pool:
    try:
        for suc,val1,val2 in pool.imap_unordered(get_lyrics, range(len(song_performer_pair))):
            if suc:
                art_song_lyrics[suc][val1] = val2
            else:
                missing_artists_song[val1].add(val2)
    except Exception as e:
        print(e)

In [191]:
pool.close()

In [186]:
temp = {}
for i in list(missing_artists_song.keys()):
    temp[i] = list(missing_artists_song[i])

In [182]:
jdata = json.dumps(temp)
output_file = open("Genesis_missing.json","w+")
output_file.write(jdata)
output_file.close()

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.

In [124]:
jdata = json.dumps(art_song_lyrics)
output_file = open("genesis_data.json","w+")
output_file.write(jdata)
output_file.close()

In [130]:
all_found_songs= [j for k in art_song_lyrics.keys() for j in list(art_song_lyrics[k].keys())]
len(all_found_songs)

23454

In [131]:
len(art_song_lyrics)

7429

In [113]:
len(missing_artists_song)

3035

In [137]:
4000 / 100

40.0

In [128]:
collections.Counter([j for i in list(missing_artists_song.keys()) for j in i.split(" ")]).most_common(20)

[('The', 653),
 ('&', 356),
 ('Featuring', 240),
 ('And', 167),
 ('With', 112),
 ('and', 68),
 ('His', 67),
 ('Orchestra', 64),
 ('Band', 64),
 ('Bobby', 44),
 ('Johnny', 42),
 ('Of', 33),
 ('Joe', 33),
 ('Paul', 31),
 ('John', 31),
 ('Billy', 30),
 ('Jimmy', 30),
 ('Brown', 28),
 ('Lee', 27),
 ('Ray', 25)]

In [6]:
song_pair_new = [[i, j] for i in list(missing_artists_song.keys()) for j in list(missing_artists_song[i])]

In [7]:
song_pair_new_reformatted = []
for p in song_pair_new:
    artist, song = p
    if "Featuring" in artist:
        artist = artist.split("Featuring")[0].strip()
    if "Feat." in artist:
        artist = artist.split("Feat.")[0].strip()
    if "featuring" in artist:
        artist = artist.split("featuring")[0].strip()
    if "&" in artist:
        artist = artist.split("&")[0].strip()
    if "and" in artist:
        artist = artist.split("and")[0].strip()
    if "And" in artist:
        artist = artist.split("And")[0].strip()
    if "With" in artist:
        artist = artist.split("With")[0].strip()
    if "with" in artist:
        artist = artist.split("with")[0].strip()
    if "The" in artist:
        artist = artist.split("The")[0].strip()
    if "the" in artist:
        artist = artist.split("the")[0].strip()
    if "(" in artist:
        artist = artist.split("(")[0].strip()
    song_pair_new_reformatted.append([artist, song])

In [8]:
song_pair_new_reformatted[1000]

['Dick', 'Tell Me']

In [12]:
def get_lyrics_updated(i):
    artist_name, song_title = song_pair_new_reformatted[i]
    params = {'q': song_title.lower() + " " + artist_name.lower()}
#     params = {'q': song_title.lower()}
    response = requests.get(search_url, params=params, headers=headers).json()
    song_info = False
    for hit in response["response"]["hits"]:
        if len(set(hit["result"]["primary_artist"]["name"].lower().split(" ")).intersection(set(artist_name.lower().split(" ")))) != 0:
            song_info = hit
            break
    if song_info:
        song_api_path = song_info["result"]["api_path"]
        song_url = base_url + song_api_path
        response = requests.get(song_url, headers=headers).json()
        path = response["response"]["song"]["path"]
        page_url = "http://genius.com" + path
        page = requests.get(page_url)
        html = BeautifulSoup(page.text, "html.parser")
        [h.extract() for h in html('script')]
        lyrics = html.find("div", class_="lyrics").get_text()
        return [artist_name, song_title, [i for i in lyrics.split("\n") if i != ""]]
    else:
        return [False, artist_name, song_title]

In [13]:
art_song_lyrics_updated = collections.defaultdict(dict)
missing_artists_song_updated = collections.defaultdict(set)
with multiprocessing.Pool(processes=multiprocessing.cpu_count()*8) as pool:
    try:
        for suc,val1,val2 in pool.imap_unordered(get_lyrics_updated, range(len(song_pair_new_reformatted))):
            if suc:
                art_song_lyrics_updated[suc][val1] = val2
            else:
                missing_artists_song_updated[val1].add(val2)
    except Exception as e:
        print(e)
    pool.close()

unhashable type: 'list'


In [16]:
len(missing_artists_song_updated)

1899

In [17]:
len(art_song_lyrics_updated)

339

In [20]:
pool.terminate()

In [20]:
all_found_songs_updated= [j for k in art_song_lyrics_updated.keys() for j in list(art_song_lyrics_updated[k].keys())]
len(all_found_songs_updated)

544

In [21]:
23454 + 544

23998